Test Wikibaseintegrator

Imports

In [64]:
import pandas
from wikibaseintegrator import WikibaseIntegrator
from wikibaseintegrator.wbi_config import config

Config

In [65]:
config['USER_AGENT'] = 'test'

DataFrame functions

In [66]:
def jsonToDataframe(path):
    with open(path, "r", encoding="utf-8") as f:
        dataframe = pandas.read_json(f, orient='index')
    return dataframe

def dataframeToJson(path, dataframe):
    with open(path, "w", encoding="utf-8") as f:
        f.write(dataframe.to_json(orient='index', indent=2))

Create WikibaseIntegrator object


In [67]:
wbi = WikibaseIntegrator()

Get a single item


In [68]:
castle = wbi.item.get('Q1049154')

Get english label

In [69]:
castle.labels.get('en').value

'Castle of Carlos V, Hondarribia'

Get Country label in french

In [70]:
def GetCountryLabelInFrench(itemID):
    item_data=wbi.item.get(itemID) # get the item global data
    country_id=item_data.claims.get('P17')[0].mainsnak.datavalue['value']['id'] # get the country id of the item
    country=wbi.item.get(country_id) # get the country global data
    return country.labels.get('fr').value # get the country label in french

Get all items from a dataframe

In [71]:
dataPath = "data.json"
dataframe = jsonToDataframe(dataPath)
items_tuples = []
for i in range(len(dataframe.index)): 
    itemID=dataframe._get_value(i,"entityID")
    items_tuples.append(itemID)
items_tuples

['Q1024829', 'Q10300408', 'Q1024943']

Add country information to the tags column

In [72]:
for i in range(len(items_tuples)):
    country = GetCountryLabelInFrench(items_tuples[i])
    dataframe.at[i, "tags"].append("#"+country)
dataframe

,label,link,entityID,format,path,size,tags,date,style
0,Buenos Aires Cabildo,http://commons.wikimedia.org/wiki/Special:File...,Q1024829,jpg,./images/Buenos Aires Cabildo.jpg,NaN,"[#monument, #16ème siècle, #Argentine]",1725,[Q840829]
1,Group of Houses of the Santa Casa de Misericórdia,http://commons.wikimedia.org/wiki/Special:File...,Q10300408,jpg,./images/Group of Houses of the Santa Casa de ...,NaN,"[#monument, #16ème siècle, #Brésil]",1612,"[Q131808, Q840829, Q1114972]"
2,Iglesia de la Concepción,http://commons.wikimedia.org/wiki/Special:File...,Q1024943,jpg,./images/Iglesia de la Concepción.jpg,NaN,"[#monument, #16ème siècle, #Espagne]",1511,[Q37853]


## Ajout du style architectural

In [73]:
def GetArchitecturalStyle(styleID):
    styleL=[]
    for i in range(len(styleID)):
        style = wbi.item.get(styleID[i])
        if style.labels.get('fr') is None:
            style_name = style.labels.get('en').value
        else:
            style_name = style.labels.get('fr').value
        styleL.append("#"+style_name)
    return styleL # get the style label in french

In [74]:
ID=["Q5911355"]
print(GetArchitecturalStyle(ID))

['#Church of San Sebastián, Villacastín']


In [75]:
dataframe._get_value(1,"style")

['Q131808', 'Q840829', 'Q1114972']

In [76]:
for k in range(len(dataframe.index)):
    style = GetArchitecturalStyle(dataframe._get_value(k,"style"))
    for i in range(len(style)):
        dataframe.at[k, "tags"].append(style[i])
dataframe

,label,link,entityID,format,path,size,tags,date,style
0,Buenos Aires Cabildo,http://commons.wikimedia.org/wiki/Special:File...,Q1024829,jpg,./images/Buenos Aires Cabildo.jpg,NaN,"[#monument, #16ème siècle, #Argentine, #archit...",1725,[Q840829]
1,Group of Houses of the Santa Casa de Misericórdia,http://commons.wikimedia.org/wiki/Special:File...,Q10300408,jpg,./images/Group of Houses of the Santa Casa de ...,NaN,"[#monument, #16ème siècle, #Brésil, #maniérism...",1612,"[Q131808, Q840829, Q1114972]"
2,Iglesia de la Concepción,http://commons.wikimedia.org/wiki/Special:File...,Q1024943,jpg,./images/Iglesia de la Concepción.jpg,NaN,"[#monument, #16ème siècle, #Espagne, #baroque]",1511,[Q37853]


In [77]:
#dataframeToJson(dataPath, dataframe)